In [1]:
import AI_Frameworks_LDA
from tqdm import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/osama.nasir@ebryx.com/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import nltk
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# sklearn
from sklearn.preprocessing import MinMaxScaler

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
#%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
#df_tmp = curriculaLDA.loadCurricula()

In [4]:
#df_tmp.to_excel("all_curricula.xlsx", engine='xlsxwriter')
#len(df_tmp)

# Preprocessing Data

In [5]:
df1 = pd.read_excel('SDG_Project_text.xlsx')
#df = df_tmp

In [6]:
df1.head()

,Project Name,text
0,DYMAXION LABS,Dymaxion Labs facilitates our understanding of...
1,Layers against inequality,Paraguay has a broad rural social group with m...
2,ParaEmpleo,ParaEmpleo is the first digital platform in th...
3,Support to Workers and Migrants,Given a migrant and a job on the Internet that...
4,MERON,"Kimetrica, a social enterprise with bases in t..."


In [7]:
len(df1)

108

In [8]:
df2 = pd.read_excel('SDG_Project_meta.xlsx')

In [9]:
df2.head()

,Project Name,SDG1,SDG2,SDG3,SDG4,SDG5,SDG6,SDG7,SDG8,SDG9,SDG10,SDG11,SDG12,SDG13,SDG14,SDG15,SDG16,SDG17
0,DYMAXION LABS,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Layers against inequality,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
2,ParaEmpleo,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
3,Support to Workers and Migrants,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0
4,MERON,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
len(df2)

108

In [11]:
corpus, id2word, data_lemmatized = AI_Frameworks_LDA.preprocessData(df1)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/osama.nasir@ebryx.com/anaconda3/envs/myenv/lib/python3.7/site-packages/en_core_web_sm
-->
/home/osama.nasir@ebryx.com/anaconda3/envs/myenv/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [12]:
print(len(corpus))
print(len(id2word))
print(len(data_lemmatized))
#print(data_lemmatized)

108
1471
108


In [13]:
vocab = list()
for i in range(len(id2word)):
    vocab.append(id2word[i])

In [14]:
with open('vocab.txt', 'w') as filehandle:
    for listitem in vocab:
        filehandle.write('%s\n' % str(listitem))

# SDG model

In [15]:
#defining all 17 SDGs
sdg1 = ['africa', 'basic', 'class', 'countries', 'developing', 'disadvantaged', 'discrimination', 'distribution', 'economic', 'end', 'environment', 'equality', 'eradication', 'extreme', 'financial', 'inclusion', 'income', 'life', 'line', 'microfinance', 'poor', 'poverty', 'protection', 'quality', 'resources', 'services', 'social', 'sustainable', 'systems', 'third', 'vulnerable', 'wealth', 'world']
sdg2 = ['agriculture', 'diversity', 'food', 'genetic', 'genetics', 'growth', 'health', 'hunger', 'hungry', 'improved', 'infrastructure', 'innovations', 'legumes', 'life', 'maize', 'malnourished', 'malnutrition', 'needs', 'nourished', 'nutrition', 'nutritional', 'nutritious', 'people', 'poverty', 'produce', 'producers', 'production', 'productivity', 'quality', 'resilient', 'restrictions', 'rural', 'scale', 'seeds', 'small', 'stunted', 'stunting', 'sufficient', 'sustainable', 'trade', 'under', 'undernourished', 'wasting', 'world']
sdg3 = ['abuse', 'access', 'accidents', 'affordable', 'aids', 'air', 'alcohol', 'antenatal', 'antiretroviral', 'autonomy', 'biomedical', 'bodily', 'borne', 'care', 'child', 'clean', 'constrained', 'contamination', 'contraceptive', 'control', 'countries', 'coverage', 'death', 'deaths', 'density', 'dental', 'developing', 'disability', 'disease', 'diseases', 'drug', 'expectancy', 'family', 'health', 'healthcare', 'healthy', 'hepatitis', 'hiv', 'hygiene', 'improving', 'inclusion', 'increasing', 'indigenous', 'infected', 'international', 'life', 'lives', 'location', 'malaria', 'maternal', 'measles', 'medical', 'medicines', 'mental', 'mortality', 'narcotic', 'neonatal', 'organisation', 'planning', 'policy', 'polio', 'politics', 'pollution', 'premature', 'preventable', 'reducing', 'refugees', 'regulations', 'reproductive', 'resource', 'road', 'sanitation', 'services', 'sexual', 'soil', 'substance', 'support', 'therapy', 'tobacco', 'traffic', 'treatment', 'tuberculosis', 'universal', 'use', 'vaccines', 'violence', 'wash', 'water', 'well', 'wellbeing', 'worker', 'world']
sdg4 = ['access', 'basic', 'childhood', 'citizenship', 'cooperation', 'cultural', 'developing', 'development', 'disability', 'disparities', 'disparity', 'diversity', 'education', 'enrolment', 'equal', 'equality', 'equitable', 'equity', 'gender', 'global', 'inclusion', 'inclusive', 'innovation', 'international', 'learning', 'lifelong', 'literacy', 'numeracy', 'opportunities', 'preprimary', 'primary', 'qualified', 'refugees', 'rights', 'scholarships', 'school', 'secondary', 'sensitive', 'skills', 'sustainability', 'teacher', 'teachers', 'training', 'universal', 'vocational', 'vulnerable', 'women']
sdg5 = ['access', 'against', 'basic', 'coverage', 'dignity', 'disadvantaged', 'discrimination', 'employment', 'empower', 'empowerment', 'equal', 'equality', 'exploitation', 'female', 'feminism', 'forced', 'gender', 'genital', 'girls', 'governance', 'health', 'human', 'humanitarian', 'inclusion', 'living', 'marginalised', 'marriage', 'mutilation', 'opportunities', 'parity', 'pay', 'reproductive', 'rights', 'sexual', 'social', 'standards', 'trafficking', 'universal', 'violence', 'women', 'work', 'workplace']
sdg6 = ['access', 'accessible', 'affordable', 'aquifer', 'basins', 'cities', 'clean', 'contaminated', 'defecation', 'desalination', 'diarrhoeal', 'disasters', 'diseases', 'drinking', 'drought', 'dumping', 'ecosystem', 'ecosystems', 'efficiency', 'equitable', 'floods', 'harvesting', 'hydropower', 'hygiene', 'improving', 'inadequate', 'infrastructure', 'irrigation', 'lakes', 'latrines', 'management', 'pollution', 'protection', 'quality', 'recycled', 'resources', 'restoration', 'reuse', 'river', 'rivers', 'safe', 'sanitation', 'scarcity', 'sewerage', 'supply', 'sustainable', 'third', 'toilets', 'treatment', 'untreated', 'urban', 'waste', 'wastewater', 'water', 'water_related', 'water_use', 'withdrawals', 'world']
sdg7 = ['affordable', 'alternative', 'animal', 'battery', 'carbon', 'charcoal', 'clean', 'cleaner', 'climate', 'coal', 'economy', 'efficiency', 'electricity', 'emissions', 'energy', 'fossil', 'fossil_fuel', 'fuel', 'fuels', 'gas', 'goal', 'green', 'greenhouse', 'hydroelectric', 'infrastructure', 'modern', 'power', 'reliable', 'renewable', 'research', 'services', 'solar', 'sustainable', 'technology', 'turbine', 'vehicles', 'waste', 'wave', 'wind', 'wood']
sdg8 = ['aid', 'banking', 'child', 'consumption', 'creation', 'creativity', 'culture', 'development', 'domestic', 'economic', 'economy', 'efficiency', 'employment', 'enterprises', 'entrepreneurship', 'equal', 'eradication', 'finance', 'financial', 'forced', 'gdp', 'global', 'gross', 'growth', 'human', 'inclusive', 'innovation', 'insurance', 'job', 'jobs', 'labour', 'life', 'market', 'micro', 'migrant', 'migrants', 'modern', 'opportunities', 'oriented', 'pay', 'policies', 'policy', 'poverty', 'product', 'production', 'productive', 'productivity', 'public', 'quality', 'resource', 'rights', 'safe', 'secure', 'services', 'slavery', 'social', 'society', 'soldiers', 'stable', 'sustainable', 'tourism', 'trade', 'trafficking', 'unemployment', 'paid', 'women', 'work', 'workers', 'world', 'youth']
sdg9 = ['access', 'affordable', 'banks', 'capabilities', 'chains', 'clean', 'communication', 'cooperation', 'countries', 'credit', 'data', 'developing', 'development', 'diversification', 'economic', 'education', 'efficiency', 'electrical', 'energy', 'enterprises', 'environmentally', 'financial', 'ict', 'industrial', 'industrialisation', 'information', 'infrastructure', 'innovation', 'internet', 'irrigation', 'knowledge', 'markets', 'mobile', 'national', 'network', 'networks', 'phone', 'policy', 'power', 'public', 'quality', 'regional', 'research', 'resilient', 'resource', 'resources', 'roads', 'sanitation', 'scientific', 'security', 'service', 'services', 'society', 'sustainable', 'technological', 'technologies', 'technology', 'trade', 'transborder', 'transport', 'value', 'water']
sdg10 = ['affordable', 'age', 'ageism', 'aid', 'assistance', 'business', 'children', 'countries', 'culture', 'developing', 'development', 'disabilities', 'discrimination', 'discriminatory', 'economy', 'education', 'empower', 'equal', 'equality', 'equity', 'ethnicity', 'financial', 'foreign', 'gender', 'global', 'growth', 'health', 'homelessness', 'homophobia', 'housing', 'human', 'inclusion', 'income', 'indigenous', 'inequalities', 'inequality', 'investment', 'markets', 'migrant', 'migration', 'nations', 'opportunity', 'policy', 'population', 'poverty', 'protection', 'public', 'quality', 'race', 'racisim', 'reduce', 'religion', 'remittance', 'rights', 'rural', 'sex', 'sexism', 'social', 'society', 'states', 'trade', 'vulnerable', 'world']
sdg11 = ['adaptable', 'adaptation', 'affordable', 'air', 'buildings', 'building', 'change', 'cities', 'city', 'climate', 'communities', 'community', 'consumption', 'crowding', 'cultural', 'decentralisation', 'development', 'disaster', 'disasters', 'efficiency', 'fine', 'generation', 'green', 'growth', 'heritage', 'housing', 'human', 'impact', 'inadequate', 'informal', 'infrastructure', 'land', 'local', 'management', 'materials', 'matter', 'mitigation', 'natural', 'particulate', 'planning', 'pollution', 'population', 'public', 'quality', 'reduction', 'resilient', 'resource', 'risk', 'road', 'safe', 'safety', 'settlements', 'shanty', 'slums', 'smart', 'solid', 'spaces', 'strategy', 'suburban', 'sustainability', 'sustainable', 'systems', 'town', 'transport', 'urban', 'urbanisation', 'waste', 'water', 'water_related']
sdg12 = ['capitalism', 'cars', 'chain', 'chains', 'circular', 'commercial', 'consumer', 'consumerism', 'consumption', 'cycle', 'decarbonisation', 'development', 'distortions', 'ecological', 'economy', 'efficiency', 'efficient', 'energy', 'enterprises', 'food', 'fossil', 'fuel', 'future', 'gasses', 'generation', 'global', 'goods', 'greenhouse', 'harvest', 'industry', 'levels', 'losses', 'management', 'market', 'materialism', 'materials', 'monitoring', 'natural', 'obsolescence', 'overconsumption', 'pollution', 'practices', 'procurement', 'production', 'proof', 'public', 'recycle', 'recycling', 'reduce', 'reduction', 'renewable', 'resource', 'resources', 'responsible', 'retail', 'reuse', 'subsidies', 'supply', 'sustainable', 'tourism', 'vehicles', 'waste', 'wasteful', 'water']
sdg13 = ['action', 'adaptation', 'agreement', 'average', 'capture', 'carbon', 'change', 'changing', 'climate', 'co2', 'conversion', 'cop', 'dioxide', 'disasters', 'disease', 'economy', 'ecosystems', 'emissions', 'events', 'extreme', 'gas', 'gases', 'gender', 'global', 'greenhouse', 'hazards', 'ice', 'impact', 'infectious', 'management', 'mitigation', 'natural', 'ocean', 'paris', 'patterns', 'planning', 'policy', 'politics', 'pollution', 'refugees', 'related', 'renewable', 'resilience', 'rise', 'rising', 'sea', 'systems', 'temperature', 'warming', 'warning', 'weather']
sdg14 = ['acidification', 'areas', 'artisanal', 'biodiversity', 'bleaching', 'carbon', 'coastal', 'coastlines', 'conserve', 'coral', 'dioxide', 'ecosystem', 'ecosystems', 'fish', 'fisheries', 'fishers', 'fishing', 'global', 'grasses', 'habitats', 'illegal', 'kelp', 'law', 'management', 'marine', 'ocean', 'oceanography', 'oceans', 'overfishing', 'parks', 'policy', 'pollution', 'practices', 'productive', 'protected', 'reef', 'resources', 'sea', 'seas', 'species', 'stocks', 'sustainable', 'temperature', 'unregulated', 'warming', 'water']
sdg15 = ['afforestation', 'agriculture', 'alien', 'animals', 'arable', 'bees', 'biodiversity', 'conservation', 'deforestation', 'degradation', 'desertification', 'drought', 'drylands', 'ecosystem', 'ecosystems', 'extinct', 'extinction', 'fauna', 'flora', 'forest', 'forests', 'genetic', 'illegal', 'illicit', 'indigenous', 'invasive', 'land', 'manage', 'managed', 'management', 'microorganisms', 'permaculture', 'plan', 'plants', 'poaching', 'populations', 'poverty', 'products', 'protected', 'reforestation', 'resources', 'restoration', 'soil', 'species', 'strategic', 'sustainability', 'terrestrial', 'threatened', 'trafficking', 'tree', 'wetlands', 'wildlife']
sdg16 = ['abuse', 'access', 'accountability', 'accountable', 'against', 'arbitrary', 'arms', 'assets', 'birth', 'bribery', 'children', 'combat', 'conflict', 'conflicts', 'corruption', 'crime', 'decision_making', 'detainees', 'detention', 'disappearance', 'discrimination', 'displaced', 'education', 'enforced', 'equal', 'equity', 'evasion', 'exploitation', 'financial', 'freedom', 'geography', 'governance', 'hate', 'human', 'identity', 'illegal', 'illicit', 'inclusion', 'inclusive', 'institutions', 'internally', 'judiciary', 'justice', 'law', 'legal', 'life', 'national', 'nonviolence', 'organized', 'paris', 'peace', 'peaceful', 'physical', 'police', 'policy', 'poverty', 'prevent', 'principles', 'psychological', 'public', 'quality', 'rates', 'registration', 'representative', 'resolution', 'rights', 'rule', 'security', 'seizures', 'sexual', 'societies', 'societies', 'society', 'stolen', 'tax', 'terrorism', 'theft', 'threats', 'torture', 'trafficking', 'transparency', 'sentenced', 'unstable', 'victims', 'violence', 'weapon', 'women']
sdg17 = ['agenda', 'agreements', 'aid', 'assistance', 'average', 'building', 'capacity', 'census', 'civil', 'communication', 'cooperation', 'countries', 'data', 'debt', 'developing', 'development', 'disaggregated', 'doha', 'entrepreneurs', 'entrepreneurship', 'environmentally', 'eradication', 'foreign', 'fostering', 'fundamental', 'global', 'housing', 'innovation', 'international', 'investments', 'knowledge', 'stakeholder', 'official', 'organization', 'partnership', 'partnerships', 'population', 'poverty', 'principles', 'public', 'private', 'science', 'sharing', 'society', 'sound', 'stability', 'statistics', 'support', 'sustainability', 'sustainable', 'tariff', 'technologies', 'technology', 'trade', 'transfer', 'weighted', 'women', 'world']
sdg_list = [sdg1, sdg2, sdg3, sdg4, sdg5, sdg6, sdg7, sdg8, sdg9, sdg10, sdg11, sdg12, sdg13, sdg14, sdg15, sdg16, sdg17]

# KEYWORD matching (word_count/doc_len)

In [16]:
#sdg bag of words
sdg_bow = [id2word.doc2bow(text) for text in sdg_list]

In [17]:
# vectorized the sdg bag of words into a vector
sdg_vectorized = np.zeros((len(sdg_list), len(id2word)))
for i, sdg_bow_i in enumerate(sdg_bow):
    for id, count in sdg_bow_i:
        sdg_vectorized[i][id] = count

In [18]:
#vectorized each document (bag of words) into a vector
corpus_vectorized = np.zeros((len(corpus), len(id2word)))
for i, corpus_i in enumerate(corpus):
    for id, count in corpus_i:
        corpus_vectorized[i][id] = count

In [19]:
#total sdg words matched in a document
sdg_word_match = np.dot(corpus_vectorized,sdg_vectorized.transpose())

#total number of words in each document
total_words = corpus_vectorized.sum(axis=1)

sdg_contr = list()
#filename_list = list()

# probability estimation
for i, corpus_i in enumerate(total_words):
    sdg_contr.append(sdg_word_match[i]/corpus_i)
    #filename_list.append(df1['Project Name'][i])
    
# round off to 4 decimal points
#sdg_contr = np.around(sdg_contr, 4)

#sdg_curricula = sdg_contr[:166,:]
#sdg_framework = sdg_contr[166:,:]
#file_curricula = filename_list[:166]
#file_framework = filename_list[166:]

# create a dataframe for sdg probabilities of each curricula with filename
project_sdg_df = pd.DataFrame(data=sdg_contr)
#project_sdg_df['FILE NAME'] = filename_list

# create a dataframe for sdg probabilities of each curricula with filename
#framework_sdg_df = pd.DataFrame(data=sdg_framework)
#framework_sdg_df['FILE NAME'] = file_framework

#rename the columns
columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']
project_sdg_df.columns = columns
#curricula_sdg_df.columns = columns

#project_sdg_df.to_excel("KW1.xlsx", engine='xlsxwriter')
#curricula_sdg_df.to_excel("curricula_sdg.xlsx", engine='xlsxwriter')

In [20]:
project_sdg_df

,sdg1,sdg2,sdg3,sdg4,sdg5,sdg6,sdg7,sdg8,sdg9,sdg10,sdg11,sdg12,sdg13,sdg14,sdg15,sdg16,sdg17
0,0.017544,0.017544,0.017544,0.000000,0.000000,0.017544,0.017544,0.070175,0.087719,0.052632,0.052632,0.052632,0.017544,0.000000,0.000000,0.052632,0.105263
1,0.044944,0.067416,0.033708,0.044944,0.056180,0.033708,0.022472,0.067416,0.044944,0.089888,0.022472,0.044944,0.011236,0.011236,0.033708,0.067416,0.022472
2,0.000000,0.000000,0.014085,0.000000,0.042254,0.000000,0.000000,0.140845,0.014085,0.014085,0.000000,0.028169,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.009852,0.019704,0.098522,0.019704,0.054187,0.024631,0.004926,0.093596,0.049261,0.029557,0.014778,0.004926,0.000000,0.000000,0.000000,0.014778,0.004926
4,0.000000,0.056338,0.056338,0.014085,0.014085,0.000000,0.028169,0.070423,0.042254,0.028169,0.014085,0.014085,0.000000,0.000000,0.000000,0.000000,0.056338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.031847,0.012739,0.101911,0.038217,0.025478,0.006369,0.012739,0.108280,0.050955,0.089172,0.025478,0.012739,0.006369,0.000000,0.000000,0.031847,0.031847
104,0.007299,0.029197,0.080292,0.014599,0.000000,0.072993,0.072993,0.029197,0.080292,0.014599,0.116788,0.043796,0.036496,0.043796,0.043796,0.000000,0.051095
105,0.000000,0.030303,0.030303,0.000000,0.000000,0.030303,0.015152,0.007576,0.060606,0.000000,0.090909,0.045455,0.060606,0.022727,0.045455,0.000000,0.045455
106,0.009615,0.019231,0.019231,0.000000,0.000000,0.000000,0.028846,0.009615,0.019231,0.000000,0.076923,0.000000,0.019231,0.009615,0.038462,0.019231,0.000000


In [21]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df.T).T)
norm_sdg_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.166667,0.166667,0.166667,0.000000,0.000000,0.166667,0.166667,0.666667,0.833333,0.500000,0.500000,0.500000,0.166667,0.000,0.000000,0.500000,1.000000
1,0.428571,0.714286,0.285714,0.428571,0.571429,0.285714,0.142857,0.714286,0.428571,1.000000,0.142857,0.428571,0.000000,0.000,0.285714,0.714286,0.142857
2,0.000000,0.000000,0.100000,0.000000,0.300000,0.000000,0.000000,1.000000,0.100000,0.100000,0.000000,0.200000,0.000000,0.000,0.000000,0.000000,0.000000
3,0.100000,0.200000,1.000000,0.200000,0.550000,0.250000,0.050000,0.950000,0.500000,0.300000,0.150000,0.050000,0.000000,0.000,0.000000,0.150000,0.050000
4,0.000000,0.800000,0.800000,0.200000,0.200000,0.000000,0.400000,1.000000,0.600000,0.400000,0.200000,0.200000,0.000000,0.000,0.000000,0.000000,0.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.294118,0.117647,0.941176,0.352941,0.235294,0.058824,0.117647,1.000000,0.470588,0.823529,0.235294,0.117647,0.058824,0.000,0.000000,0.294118,0.294118
104,0.062500,0.250000,0.687500,0.125000,0.000000,0.625000,0.625000,0.250000,0.687500,0.125000,1.000000,0.375000,0.312500,0.375,0.375000,0.000000,0.437500
105,0.000000,0.333333,0.333333,0.000000,0.000000,0.333333,0.166667,0.083333,0.666667,0.000000,1.000000,0.500000,0.666667,0.250,0.500000,0.000000,0.500000
106,0.125000,0.250000,0.250000,0.000000,0.000000,0.000000,0.375000,0.125000,0.250000,0.000000,1.000000,0.000000,0.250000,0.125,0.500000,0.250000,0.000000


In [23]:
#import numpy
#threshold = numpy.argsort(project_sdg_df)['sdg17']
df2.drop('Project Name', axis=1, inplace=True)

In [23]:
doc_num = 108
i = 0
match = 0
cutoff = 0.6

for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score: %.2f" % (match/doc_num*100) +"%")

Accuracy Score: 90.74%


# KEYWORD matching (word_match/sdg_len)

In [24]:
#sdg bag of words
sdg_bow = [id2word.doc2bow(text) for text in sdg_list]

In [25]:
# vectorized the sdg bag of words into a vector
sdg_vectorized = np.zeros((len(sdg_list), len(id2word)))
for i, sdg_bow_i in enumerate(sdg_bow):
    for id, count in sdg_bow_i:
        sdg_vectorized[i][id] = 1

In [26]:
#vectorized each document (bag of words) into a vector
corpus_vectorized = np.zeros((len(corpus), len(id2word)))
for i, corpus_i in enumerate(corpus):
    for id, count in corpus_i:
        corpus_vectorized[i][id] = 1

In [27]:
#total sdg words matched in a document
sdg_word_match = np.dot(corpus_vectorized,sdg_vectorized.transpose())

#total number of words in each document
#total_words = corpus_vectorized.sum(axis=1)
total_words = sdg_vectorized.sum(axis=1)

sdg_contr = list()
#filename_list = list()

# probability estimation
for i, corpus_i in enumerate(total_words):
    sdg_contr.append(sdg_word_match.T[i]/corpus_i)
    #filename_list.append(df1['Project Name'][i])
    
# round off to 4 decimal points
#sdg_contr = np.around(sdg_contr, 4)

#sdg_curricula = sdg_contr[:166,:]
#sdg_framework = sdg_contr[166:,:]
#file_curricula = filename_list[:166]
#file_framework = filename_list[166:]

# create a dataframe for sdg probabilities of each curricula with filename
project_sdg_df = pd.DataFrame(data=sdg_contr)
project_sdg_df = project_sdg_df.transpose()
#project_sdg_df['FILE NAME'] = filename_list

# create a dataframe for sdg probabilities of each curricula with filename
#framework_sdg_df = pd.DataFrame(data=sdg_framework)
#framework_sdg_df['FILE NAME'] = file_framework

#rename the columns
columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']
project_sdg_df.columns = columns
#curricula_sdg_df.columns = columns

#project_sdg_df.to_excel("KW2.xlsx", engine='xlsxwriter')
#curricula_sdg_df.to_excel("curricula_sdg.xlsx", engine='xlsxwriter')

In [28]:
project_sdg_df

,sdg1,sdg2,sdg3,sdg4,sdg5,sdg6,sdg7,sdg8,sdg9,sdg10,sdg11,sdg12,sdg13,sdg14,sdg15,sdg16,sdg17
0,0.05,0.043478,0.022727,0.00,0.000000,0.041667,0.045455,0.088235,0.075,0.064516,0.048780,0.066667,0.038462,0.00,0.000000,0.086957,0.129032
1,0.15,0.173913,0.045455,0.15,0.285714,0.083333,0.045455,0.117647,0.050,0.225806,0.048780,0.100000,0.038462,0.05,0.166667,0.173913,0.064516
2,0.00,0.000000,0.022727,0.00,0.071429,0.000000,0.000000,0.088235,0.025,0.032258,0.000000,0.033333,0.000000,0.00,0.000000,0.000000,0.000000
3,0.10,0.086957,0.090909,0.10,0.357143,0.125000,0.045455,0.176471,0.100,0.129032,0.048780,0.033333,0.000000,0.00,0.000000,0.043478,0.032258
4,0.00,0.130435,0.045455,0.05,0.071429,0.000000,0.090909,0.088235,0.075,0.064516,0.024390,0.033333,0.000000,0.00,0.000000,0.000000,0.129032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.10,0.086957,0.113636,0.10,0.071429,0.041667,0.090909,0.117647,0.150,0.193548,0.073171,0.033333,0.038462,0.00,0.000000,0.130435,0.129032
104,0.05,0.086957,0.045455,0.10,0.000000,0.208333,0.227273,0.117647,0.150,0.064516,0.146341,0.100000,0.076923,0.15,0.166667,0.000000,0.129032
105,0.00,0.130435,0.090909,0.00,0.000000,0.083333,0.090909,0.029412,0.150,0.000000,0.195122,0.133333,0.230769,0.05,0.166667,0.000000,0.129032
106,0.05,0.086957,0.045455,0.00,0.000000,0.000000,0.090909,0.029412,0.025,0.000000,0.121951,0.000000,0.076923,0.05,0.222222,0.086957,0.000000


In [29]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df.T).T)
norm_sdg_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.387500,0.336957,0.176136,0.000000,0.000000,0.322917,0.352273,0.683824,0.581250,0.500000,0.378049,0.516667,0.298077,0.000000,0.000000,0.673913,1.000000
1,0.451111,0.547826,0.028283,0.451111,1.000000,0.181481,0.028283,0.320261,0.046667,0.757706,0.041734,0.248889,0.000000,0.046667,0.518519,0.547826,0.105376
2,0.000000,0.000000,0.257576,0.000000,0.809524,0.000000,0.000000,1.000000,0.283333,0.365591,0.000000,0.377778,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.280000,0.243478,0.254545,0.280000,1.000000,0.350000,0.127273,0.494118,0.280000,0.361290,0.136585,0.093333,0.000000,0.000000,0.000000,0.121739,0.090323
4,0.000000,1.000000,0.348485,0.383333,0.547619,0.000000,0.696970,0.676471,0.575000,0.494624,0.186992,0.255556,0.000000,0.000000,0.000000,0.000000,0.989247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.516667,0.449275,0.587121,0.516667,0.369048,0.215278,0.469697,0.607843,0.775000,1.000000,0.378049,0.172222,0.198718,0.000000,0.000000,0.673913,0.666667
104,0.220000,0.382609,0.200000,0.440000,0.000000,0.916667,1.000000,0.517647,0.660000,0.283871,0.643902,0.440000,0.338462,0.660000,0.733333,0.000000,0.567742
105,0.000000,0.565217,0.393939,0.000000,0.000000,0.361111,0.393939,0.127451,0.650000,0.000000,0.845528,0.577778,1.000000,0.216667,0.722222,0.000000,0.559140
106,0.225000,0.391304,0.204545,0.000000,0.000000,0.000000,0.409091,0.132353,0.112500,0.000000,0.548780,0.000000,0.346154,0.225000,1.000000,0.391304,0.000000


In [30]:
#import numpy
#threshold = numpy.argsort(project_sdg_df)['sdg17']

In [31]:
doc_num = 108
i = 0
match = 0

for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score: %.2f"% (match/doc_num*100)+"%")

Accuracy Score: 83.33%


# KEYWORD matching (word_count/sdg_len)

In [32]:
#sdg bag of words
sdg_bow = [id2word.doc2bow(text) for text in sdg_list]

In [33]:
# vectorized the sdg bag of words into a vector
sdg_vectorized = np.zeros((len(sdg_list), len(id2word)))
for i, sdg_bow_i in enumerate(sdg_bow):
    for id, count in sdg_bow_i:
        sdg_vectorized[i][id] = count

In [34]:
#vectorized each document (bag of words) into a vector
corpus_vectorized = np.zeros((len(corpus), len(id2word)))
for i, corpus_i in enumerate(corpus):
    for id, count in corpus_i:
        corpus_vectorized[i][id] = count

In [35]:
#total sdg words matched in a document
sdg_word_match = np.dot(corpus_vectorized,sdg_vectorized.transpose())

#total number of words in each document
#total_words = corpus_vectorized.sum(axis=1)
total_words = sdg_vectorized.sum(axis=1)

sdg_contr = list()
#filename_list = list()

# probability estimation
for i, corpus_i in enumerate(total_words):
    sdg_contr.append(sdg_word_match.T[i]/corpus_i)
    #filename_list.append(df1['Project Name'][i])
    
# round off to 4 decimal points
#sdg_contr = np.around(sdg_contr, 4)

#sdg_curricula = sdg_contr[:166,:]
#sdg_framework = sdg_contr[166:,:]
#file_curricula = filename_list[:166]
#file_framework = filename_list[166:]

# create a dataframe for sdg probabilities of each curricula with filename
project_sdg_df = pd.DataFrame(data=sdg_contr)
project_sdg_df = project_sdg_df.transpose()
#project_sdg_df['FILE NAME'] = filename_list

# create a dataframe for sdg probabilities of each curricula with filename
#framework_sdg_df = pd.DataFrame(data=sdg_framework)
#framework_sdg_df['FILE NAME'] = file_framework

#rename the columns
columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']
project_sdg_df.columns = columns
#curricula_sdg_df.columns = columns

#project_sdg_df.to_excel("KW3.xlsx", engine='xlsxwriter')
#curricula_sdg_df.to_excel("curricula_sdg.xlsx", engine='xlsxwriter')

In [36]:
project_sdg_df

,sdg1,sdg2,sdg3,sdg4,sdg5,sdg6,sdg7,sdg8,sdg9,sdg10,sdg11,sdg12,sdg13,sdg14,sdg15,sdg16,sdg17
0,0.05,0.043478,0.022727,0.00,0.000000,0.041667,0.045455,0.117647,0.125,0.096774,0.073171,0.100000,0.038462,0.00,0.000000,0.130435,0.193548
1,0.20,0.260870,0.068182,0.20,0.357143,0.125000,0.090909,0.176471,0.100,0.258065,0.048780,0.133333,0.038462,0.05,0.166667,0.260870,0.064516
2,0.00,0.000000,0.022727,0.00,0.214286,0.000000,0.000000,0.294118,0.025,0.032258,0.000000,0.066667,0.000000,0.00,0.000000,0.000000,0.000000
3,0.10,0.173913,0.454545,0.20,0.785714,0.208333,0.045455,0.558824,0.250,0.193548,0.073171,0.033333,0.000000,0.00,0.000000,0.130435,0.032258
4,0.00,0.173913,0.090909,0.05,0.071429,0.000000,0.090909,0.147059,0.075,0.064516,0.024390,0.033333,0.000000,0.00,0.000000,0.000000,0.129032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.25,0.086957,0.363636,0.30,0.285714,0.041667,0.090909,0.500000,0.200,0.451613,0.097561,0.066667,0.038462,0.00,0.000000,0.217391,0.161290
104,0.05,0.173913,0.250000,0.10,0.000000,0.416667,0.454545,0.117647,0.275,0.064516,0.390244,0.200000,0.192308,0.30,0.333333,0.000000,0.225806
105,0.00,0.173913,0.090909,0.00,0.000000,0.166667,0.090909,0.029412,0.200,0.000000,0.292683,0.200000,0.307692,0.15,0.333333,0.000000,0.193548
106,0.05,0.086957,0.045455,0.00,0.000000,0.000000,0.136364,0.029412,0.050,0.000000,0.195122,0.000000,0.076923,0.05,0.222222,0.086957,0.000000


In [37]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df.T).T)
norm_sdg_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.258333,0.224638,0.117424,0.000000,0.000000,0.215278,0.234848,0.607843,0.645833,0.500000,0.378049,0.516667,0.198718,0.000000,0.000000,0.673913,1.000000
1,0.506897,0.697901,0.093260,0.506897,1.000000,0.271552,0.164577,0.433063,0.193103,0.689099,0.032380,0.297701,0.000000,0.036207,0.402299,0.697901,0.081758
2,0.000000,0.000000,0.077273,0.000000,0.728571,0.000000,0.000000,1.000000,0.085000,0.109677,0.000000,0.226667,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.127273,0.221344,0.578512,0.254545,1.000000,0.265152,0.057851,0.711230,0.318182,0.246334,0.093126,0.042424,0.000000,0.000000,0.000000,0.166008,0.041056
4,0.000000,1.000000,0.522727,0.287500,0.410714,0.000000,0.522727,0.845588,0.431250,0.370968,0.140244,0.191667,0.000000,0.000000,0.000000,0.000000,0.741935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.500000,0.173913,0.727273,0.600000,0.571429,0.083333,0.181818,1.000000,0.400000,0.903226,0.195122,0.133333,0.076923,0.000000,0.000000,0.434783,0.322581
104,0.110000,0.382609,0.550000,0.220000,0.000000,0.916667,1.000000,0.258824,0.605000,0.141935,0.858537,0.440000,0.423077,0.660000,0.733333,0.000000,0.496774
105,0.000000,0.521739,0.272727,0.000000,0.000000,0.500000,0.272727,0.088235,0.600000,0.000000,0.878049,0.600000,0.923077,0.450000,1.000000,0.000000,0.580645
106,0.225000,0.391304,0.204545,0.000000,0.000000,0.000000,0.613636,0.132353,0.225000,0.000000,0.878049,0.000000,0.346154,0.225000,1.000000,0.391304,0.000000


In [38]:
#import numpy
#threshold = numpy.argsort(project_sdg_df)['sdg17']

In [39]:
doc_num = 108
i = 0
match = 0

for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score: %.2f" % (match/doc_num*100) +"%")

Accuracy Score: 82.41%


# KEYWORD matching (word_count/sdg_len*doc_len)

In [40]:
#sdg bag of words
sdg_bow = [id2word.doc2bow(text) for text in sdg_list]

In [41]:
# vectorized the sdg bag of words into a vector
sdg_vectorized = np.zeros((len(sdg_list), len(id2word)))
for i, sdg_bow_i in enumerate(sdg_bow):
    for id, count in sdg_bow_i:
        sdg_vectorized[i][id] = count

In [42]:
#vectorized each document (bag of words) into a vector
corpus_vectorized = np.zeros((len(corpus), len(id2word)))
for i, corpus_i in enumerate(corpus):
    for id, count in corpus_i:
        corpus_vectorized[i][id] = count

In [43]:
#total sdg words matched in a document
sdg_word_match = np.dot(corpus_vectorized,sdg_vectorized.transpose())

#total number of words in each document
doc_words = corpus_vectorized.sum(axis=1)
sdg_words = sdg_vectorized.sum(axis=1)

sdg_contr = list()
#filename_list = list()

# probability estimation
for i, corpus_i in enumerate(sdg_words):
    sdg_contr.append(sdg_word_match.T[i]/corpus_i)

# create a dataframe for sdg probabilities of each curricula with filename
project_sdg_df = pd.DataFrame(data=sdg_contr)
project_sdg_df = project_sdg_df.transpose()
project_sdg_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.05,0.043478,0.022727,0.00,0.000000,0.041667,0.045455,0.117647,0.125,0.096774,0.073171,0.100000,0.038462,0.00,0.000000,0.130435,0.193548
1,0.20,0.260870,0.068182,0.20,0.357143,0.125000,0.090909,0.176471,0.100,0.258065,0.048780,0.133333,0.038462,0.05,0.166667,0.260870,0.064516
2,0.00,0.000000,0.022727,0.00,0.214286,0.000000,0.000000,0.294118,0.025,0.032258,0.000000,0.066667,0.000000,0.00,0.000000,0.000000,0.000000
3,0.10,0.173913,0.454545,0.20,0.785714,0.208333,0.045455,0.558824,0.250,0.193548,0.073171,0.033333,0.000000,0.00,0.000000,0.130435,0.032258
4,0.00,0.173913,0.090909,0.05,0.071429,0.000000,0.090909,0.147059,0.075,0.064516,0.024390,0.033333,0.000000,0.00,0.000000,0.000000,0.129032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.25,0.086957,0.363636,0.30,0.285714,0.041667,0.090909,0.500000,0.200,0.451613,0.097561,0.066667,0.038462,0.00,0.000000,0.217391,0.161290
104,0.05,0.173913,0.250000,0.10,0.000000,0.416667,0.454545,0.117647,0.275,0.064516,0.390244,0.200000,0.192308,0.30,0.333333,0.000000,0.225806
105,0.00,0.173913,0.090909,0.00,0.000000,0.166667,0.090909,0.029412,0.200,0.000000,0.292683,0.200000,0.307692,0.15,0.333333,0.000000,0.193548
106,0.05,0.086957,0.045455,0.00,0.000000,0.000000,0.136364,0.029412,0.050,0.000000,0.195122,0.000000,0.076923,0.05,0.222222,0.086957,0.000000


In [44]:
sdg_contr = list()
for i, corpus_i in enumerate(doc_words):
    sdg_contr.append(list(project_sdg_df.iloc[i]/corpus_i))
    
project_sdg_df = pd.DataFrame(data=sdg_contr)

#rename the columns
columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']
project_sdg_df.columns = columns
project_sdg_df


#project_sdg_df.to_excel("KW4.xlsx", engine='xlsxwriter')

,sdg1,sdg2,sdg3,sdg4,sdg5,sdg6,sdg7,sdg8,sdg9,sdg10,sdg11,sdg12,sdg13,sdg14,sdg15,sdg16,sdg17
0,0.000877,0.000763,0.000399,0.000000,0.000000,0.000731,0.000797,0.002064,0.002193,0.001698,0.001284,0.001754,0.000675,0.000000,0.000000,0.002288,0.003396
1,0.002247,0.002931,0.000766,0.002247,0.004013,0.001404,0.001021,0.001983,0.001124,0.002900,0.000548,0.001498,0.000432,0.000562,0.001873,0.002931,0.000725
2,0.000000,0.000000,0.000320,0.000000,0.003018,0.000000,0.000000,0.004143,0.000352,0.000454,0.000000,0.000939,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000493,0.000857,0.002239,0.000985,0.003871,0.001026,0.000224,0.002753,0.001232,0.000953,0.000360,0.000164,0.000000,0.000000,0.000000,0.000643,0.000159
4,0.000000,0.002449,0.001280,0.000704,0.001006,0.000000,0.001280,0.002071,0.001056,0.000909,0.000344,0.000469,0.000000,0.000000,0.000000,0.000000,0.001817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.001592,0.000554,0.002316,0.001911,0.001820,0.000265,0.000579,0.003185,0.001274,0.002877,0.000621,0.000425,0.000245,0.000000,0.000000,0.001385,0.001027
104,0.000365,0.001269,0.001825,0.000730,0.000000,0.003041,0.003318,0.000859,0.002007,0.000471,0.002848,0.001460,0.001404,0.002190,0.002433,0.000000,0.001648
105,0.000000,0.001318,0.000689,0.000000,0.000000,0.001263,0.000689,0.000223,0.001515,0.000000,0.002217,0.001515,0.002331,0.001136,0.002525,0.000000,0.001466
106,0.000481,0.000836,0.000437,0.000000,0.000000,0.000000,0.001311,0.000283,0.000481,0.000000,0.001876,0.000000,0.000740,0.000481,0.002137,0.000836,0.000000


In [45]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df.T).T)
norm_sdg_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.258333,0.224638,0.117424,0.000000,0.000000,0.215278,0.234848,0.607843,0.645833,0.500000,0.378049,0.516667,0.198718,0.000000,0.000000,0.673913,1.000000
1,0.506897,0.697901,0.093260,0.506897,1.000000,0.271552,0.164577,0.433063,0.193103,0.689099,0.032380,0.297701,0.000000,0.036207,0.402299,0.697901,0.081758
2,0.000000,0.000000,0.077273,0.000000,0.728571,0.000000,0.000000,1.000000,0.085000,0.109677,0.000000,0.226667,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.127273,0.221344,0.578512,0.254545,1.000000,0.265152,0.057851,0.711230,0.318182,0.246334,0.093126,0.042424,0.000000,0.000000,0.000000,0.166008,0.041056
4,0.000000,1.000000,0.522727,0.287500,0.410714,0.000000,0.522727,0.845588,0.431250,0.370968,0.140244,0.191667,0.000000,0.000000,0.000000,0.000000,0.741935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.500000,0.173913,0.727273,0.600000,0.571429,0.083333,0.181818,1.000000,0.400000,0.903226,0.195122,0.133333,0.076923,0.000000,0.000000,0.434783,0.322581
104,0.110000,0.382609,0.550000,0.220000,0.000000,0.916667,1.000000,0.258824,0.605000,0.141935,0.858537,0.440000,0.423077,0.660000,0.733333,0.000000,0.496774
105,0.000000,0.521739,0.272727,0.000000,0.000000,0.500000,0.272727,0.088235,0.600000,0.000000,0.878049,0.600000,0.923077,0.450000,1.000000,0.000000,0.580645
106,0.225000,0.391304,0.204545,0.000000,0.000000,0.000000,0.613636,0.132353,0.225000,0.000000,0.878049,0.000000,0.346154,0.225000,1.000000,0.391304,0.000000


In [46]:
#import numpy
#threshold = numpy.argsort(project_sdg_df)['sdg17']

In [47]:
doc_num = 108
i = 0
match = 0

for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score: %.2f" % (match/doc_num*100) +"%")

Accuracy Score: 82.41%


# Similarity (1-gram) Threshold=0.6

In [16]:
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-100")

In [17]:
#defining all 17 SDGs
sdg1 = ['africa', 'basic', 'class', 'countries', 'developing', 'disadvantaged', 'discrimination', 'distribution', 'economic', 'end', 'environment', 'equality', 'eradication', 'extreme', 'financial', 'inclusion', 'income', 'life', 'line', 'microfinance', 'poor', 'poverty', 'protection', 'quality', 'resources', 'services', 'social', 'sustainable', 'systems', 'third', 'vulnerable', 'wealth', 'world']
sdg2 = ['agriculture', 'diversity', 'food', 'genetic', 'genetics', 'growth', 'health', 'hunger', 'hungry', 'improved', 'infrastructure', 'innovations', 'legumes', 'life', 'maize', 'malnourished', 'malnutrition', 'needs', 'nourished', 'nutrition', 'nutritional', 'nutritious', 'people', 'poverty', 'produce', 'producers', 'production', 'productivity', 'quality', 'resilient', 'restrictions', 'rural', 'scale', 'seeds', 'small', 'stunted', 'stunting', 'sufficient', 'sustainable', 'trade', 'under', 'undernourished', 'wasting', 'world']
sdg3 = ['abuse', 'access', 'accidents', 'affordable', 'aids', 'air', 'alcohol', 'antenatal', 'antiretroviral', 'autonomy', 'biomedical', 'bodily', 'borne', 'care', 'child', 'clean', 'constrained', 'contamination', 'contraceptive', 'control', 'countries', 'coverage', 'death', 'deaths', 'density', 'dental', 'developing', 'disability', 'disease', 'diseases', 'drug', 'expectancy', 'family', 'health', 'healthcare', 'healthy', 'hepatitis', 'hiv', 'hygiene', 'improving', 'inclusion', 'increasing', 'indigenous', 'infected', 'international', 'life', 'lives', 'location', 'malaria', 'maternal', 'measles', 'medical', 'medicines', 'mental', 'mortality', 'narcotic', 'neonatal', 'organisation', 'planning', 'policy', 'polio', 'politics', 'pollution', 'premature', 'preventable', 'reducing', 'refugees', 'regulations', 'reproductive', 'resource', 'road', 'sanitation', 'services', 'sexual', 'soil', 'substance', 'support', 'therapy', 'tobacco', 'traffic', 'treatment', 'tuberculosis', 'universal', 'use', 'vaccines', 'violence', 'wash', 'water', 'well', 'wellbeing', 'worker', 'world']
sdg4 = ['access', 'basic', 'childhood', 'citizenship', 'cooperation', 'cultural', 'developing', 'development', 'disability', 'disparities', 'disparity', 'diversity', 'education', 'enrolment', 'equal', 'equality', 'equitable', 'equity', 'gender', 'global', 'inclusion', 'inclusive', 'innovation', 'international', 'learning', 'lifelong', 'literacy', 'numeracy', 'opportunities', 'preprimary', 'primary', 'qualified', 'refugees', 'rights', 'scholarships', 'school', 'secondary', 'sensitive', 'skills', 'sustainability', 'teacher', 'teachers', 'training', 'universal', 'vocational', 'vulnerable', 'women']
sdg5 = ['access', 'against', 'basic', 'coverage', 'dignity', 'disadvantaged', 'discrimination', 'employment', 'empower', 'empowerment', 'equal', 'equality', 'exploitation', 'female', 'feminism', 'forced', 'gender', 'genital', 'girls', 'governance', 'health', 'human', 'humanitarian', 'inclusion', 'living', 'marginalised', 'marriage', 'mutilation', 'opportunities', 'parity', 'pay', 'reproductive', 'rights', 'sexual', 'social', 'standards', 'trafficking', 'universal', 'violence', 'women', 'work', 'workplace']
sdg6 = ['access', 'accessible', 'affordable', 'aquifer', 'basins', 'cities', 'clean', 'contaminated', 'defecation', 'desalination', 'diarrhoeal', 'disasters', 'diseases', 'drinking', 'drought', 'dumping', 'ecosystem', 'ecosystems', 'efficiency', 'equitable', 'floods', 'harvesting', 'hydropower', 'hygiene', 'improving', 'inadequate', 'infrastructure', 'irrigation', 'lakes', 'latrines', 'management', 'pollution', 'protection', 'quality', 'recycled', 'resources', 'restoration', 'reuse', 'river', 'rivers', 'safe', 'sanitation', 'scarcity', 'sewerage', 'supply', 'sustainable', 'third', 'toilets', 'treatment', 'untreated', 'urban', 'waste', 'wastewater', 'water', 'water_related', 'water_use', 'withdrawals', 'world']
sdg7 = ['affordable', 'alternative', 'animal', 'battery', 'carbon', 'charcoal', 'clean', 'cleaner', 'climate', 'coal', 'economy', 'efficiency', 'electricity', 'emissions', 'energy', 'fossil', 'fossil_fuel', 'fuel', 'fuels', 'gas', 'goal', 'green', 'greenhouse', 'hydroelectric', 'infrastructure', 'modern', 'power', 'reliable', 'renewable', 'research', 'services', 'solar', 'sustainable', 'technology', 'turbine', 'vehicles', 'waste', 'wave', 'wind', 'wood']
sdg8 = ['aid', 'banking', 'child', 'consumption', 'creation', 'creativity', 'culture', 'development', 'domestic', 'economic', 'economy', 'efficiency', 'employment', 'enterprises', 'entrepreneurship', 'equal', 'eradication', 'finance', 'financial', 'forced', 'gdp', 'global', 'gross', 'growth', 'human', 'inclusive', 'innovation', 'insurance', 'job', 'jobs', 'labour', 'life', 'market', 'micro', 'migrant', 'migrants', 'modern', 'opportunities', 'oriented', 'pay', 'policies', 'policy', 'poverty', 'product', 'production', 'productive', 'productivity', 'public', 'quality', 'resource', 'rights', 'safe', 'secure', 'services', 'slavery', 'social', 'society', 'soldiers', 'stable', 'sustainable', 'tourism', 'trade', 'trafficking', 'unemployment', 'paid', 'women', 'work', 'workers', 'world', 'youth']
sdg9 = ['access', 'affordable', 'banks', 'capabilities', 'chains', 'clean', 'communication', 'cooperation', 'countries', 'credit', 'data', 'developing', 'development', 'diversification', 'economic', 'education', 'efficiency', 'electrical', 'energy', 'enterprises', 'environmentally', 'financial', 'ict', 'industrial', 'industrialisation', 'information', 'infrastructure', 'innovation', 'internet', 'irrigation', 'knowledge', 'markets', 'mobile', 'national', 'network', 'networks', 'phone', 'policy', 'power', 'public', 'quality', 'regional', 'research', 'resilient', 'resource', 'resources', 'roads', 'sanitation', 'scientific', 'security', 'service', 'services', 'society', 'sustainable', 'technological', 'technologies', 'technology', 'trade', 'transborder', 'transport', 'value', 'water']
sdg10 = ['affordable', 'age', 'ageism', 'aid', 'assistance', 'business', 'children', 'countries', 'culture', 'developing', 'development', 'disabilities', 'discrimination', 'discriminatory', 'economy', 'education', 'empower', 'equal', 'equality', 'equity', 'ethnicity', 'financial', 'foreign', 'gender', 'global', 'growth', 'health', 'homelessness', 'homophobia', 'housing', 'human', 'inclusion', 'income', 'indigenous', 'inequalities', 'inequality', 'investment', 'markets', 'migrant', 'migration', 'nations', 'opportunity', 'policy', 'population', 'poverty', 'protection', 'public', 'quality', 'race', 'racisim', 'reduce', 'religion', 'remittance', 'rights', 'rural', 'sex', 'sexism', 'social', 'society', 'states', 'trade', 'vulnerable', 'world']
sdg11 = ['adaptable', 'adaptation', 'affordable', 'air', 'buildings', 'building', 'change', 'cities', 'city', 'climate', 'communities', 'community', 'consumption', 'crowding', 'cultural', 'decentralisation', 'development', 'disaster', 'disasters', 'efficiency', 'fine', 'generation', 'green', 'growth', 'heritage', 'housing', 'human', 'impact', 'inadequate', 'informal', 'infrastructure', 'land', 'local', 'management', 'materials', 'matter', 'mitigation', 'natural', 'particulate', 'planning', 'pollution', 'population', 'public', 'quality', 'reduction', 'resilient', 'resource', 'risk', 'road', 'safe', 'safety', 'settlements', 'shanty', 'slums', 'smart', 'solid', 'spaces', 'strategy', 'suburban', 'sustainability', 'sustainable', 'systems', 'town', 'transport', 'urban', 'urbanisation', 'waste', 'water', 'water_related']
sdg12 = ['capitalism', 'cars', 'chain', 'chains', 'circular', 'commercial', 'consumer', 'consumerism', 'consumption', 'cycle', 'decarbonisation', 'development', 'distortions', 'ecological', 'economy', 'efficiency', 'efficient', 'energy', 'enterprises', 'food', 'fossil', 'fuel', 'future', 'gasses', 'generation', 'global', 'goods', 'greenhouse', 'harvest', 'industry', 'levels', 'losses', 'management', 'market', 'materialism', 'materials', 'monitoring', 'natural', 'obsolescence', 'overconsumption', 'pollution', 'practices', 'procurement', 'production', 'proof', 'public', 'recycle', 'recycling', 'reduce', 'reduction', 'renewable', 'resource', 'resources', 'responsible', 'retail', 'reuse', 'subsidies', 'supply', 'sustainable', 'tourism', 'vehicles', 'waste', 'wasteful', 'water']
sdg13 = ['action', 'adaptation', 'agreement', 'average', 'capture', 'carbon', 'change', 'changing', 'climate', 'co2', 'conversion', 'cop', 'dioxide', 'disasters', 'disease', 'economy', 'ecosystems', 'emissions', 'events', 'extreme', 'gas', 'gases', 'gender', 'global', 'greenhouse', 'hazards', 'ice', 'impact', 'infectious', 'management', 'mitigation', 'natural', 'ocean', 'paris', 'patterns', 'planning', 'policy', 'politics', 'pollution', 'refugees', 'related', 'renewable', 'resilience', 'rise', 'rising', 'sea', 'systems', 'temperature', 'warming', 'warning', 'weather']
sdg14 = ['acidification', 'areas', 'artisanal', 'biodiversity', 'bleaching', 'carbon', 'coastal', 'coastlines', 'conserve', 'coral', 'dioxide', 'ecosystem', 'ecosystems', 'fish', 'fisheries', 'fishers', 'fishing', 'global', 'grasses', 'habitats', 'illegal', 'kelp', 'law', 'management', 'marine', 'ocean', 'oceanography', 'oceans', 'overfishing', 'parks', 'policy', 'pollution', 'practices', 'productive', 'protected', 'reef', 'resources', 'sea', 'seas', 'species', 'stocks', 'sustainable', 'temperature', 'unregulated', 'warming', 'water']
sdg15 = ['afforestation', 'agriculture', 'alien', 'animals', 'arable', 'bees', 'biodiversity', 'conservation', 'deforestation', 'degradation', 'desertification', 'drought', 'drylands', 'ecosystem', 'ecosystems', 'extinct', 'extinction', 'fauna', 'flora', 'forest', 'forests', 'genetic', 'illegal', 'illicit', 'indigenous', 'invasive', 'land', 'manage', 'managed', 'management', 'microorganisms', 'permaculture', 'plan', 'plants', 'poaching', 'populations', 'poverty', 'products', 'protected', 'reforestation', 'resources', 'restoration', 'soil', 'species', 'strategic', 'sustainability', 'terrestrial', 'threatened', 'trafficking', 'tree', 'wetlands', 'wildlife']
sdg16 = ['abuse', 'access', 'accountability', 'accountable', 'against', 'arbitrary', 'arms', 'assets', 'birth', 'bribery', 'children', 'combat', 'conflict', 'conflicts', 'corruption', 'crime', 'decision_making', 'detainees', 'detention', 'disappearance', 'discrimination', 'displaced', 'education', 'enforced', 'equal', 'equity', 'evasion', 'exploitation', 'financial', 'freedom', 'geography', 'governance', 'hate', 'human', 'identity', 'illegal', 'illicit', 'inclusion', 'inclusive', 'institutions', 'internally', 'judiciary', 'justice', 'law', 'legal', 'life', 'national', 'nonviolence', 'organized', 'paris', 'peace', 'peaceful', 'physical', 'police', 'policy', 'poverty', 'prevent', 'principles', 'psychological', 'public', 'quality', 'rates', 'registration', 'representative', 'resolution', 'rights', 'rule', 'security', 'seizures', 'sexual', 'societies', 'societies', 'society', 'stolen', 'tax', 'terrorism', 'theft', 'threats', 'torture', 'trafficking', 'transparency', 'sentenced', 'unstable', 'victims', 'violence', 'weapon', 'women']
sdg17 = ['agenda', 'agreements', 'aid', 'assistance', 'average', 'building', 'capacity', 'census', 'civil', 'communication', 'cooperation', 'countries', 'data', 'debt', 'developing', 'development', 'disaggregated', 'doha', 'entrepreneurs', 'entrepreneurship', 'environmentally', 'eradication', 'foreign', 'fostering', 'fundamental', 'global', 'housing', 'innovation', 'international', 'investments', 'knowledge', 'stakeholder', 'official', 'organization', 'partnership', 'partnerships', 'population', 'poverty', 'principles', 'public', 'private', 'science', 'sharing', 'society', 'sound', 'stability', 'statistics', 'support', 'sustainability', 'sustainable', 'tariff', 'technologies', 'technology', 'trade', 'transfer', 'weighted', 'women', 'world']
sdg_list = [sdg1, sdg2, sdg3, sdg4, sdg5, sdg6, sdg7, sdg8, sdg9, sdg10, sdg11, sdg12, sdg13, sdg14, sdg15, sdg16, sdg17]

In [50]:
documents = data_lemmatized
sdg_l = sdg_list

score_vector1 = np.zeros((len(documents), len(sdg_l)))
score_vector2 = np.zeros((len(documents), len(sdg_l)))
score_vector3 = np.zeros((len(documents), len(sdg_l)))
score_vector4 = np.zeros((len(documents), len(sdg_l)))

for d, doc in tqdm(enumerate(documents)):
    d_len = len(documents[d])
    #print(d)
    for s, sdg in enumerate(sdg_l):
        s_len = len(sdg_l[s])
        score = 0
        for keyword in sdg:
            for word in doc:
                try:
                    sim = model.similarity(word, keyword)
                    if (sim >= 0.6):
                        score += sim 
                except:
                    pass
        score_vector1[d][s] = score
        score_vector2[d][s] = score / d_len
        score_vector3[d][s] = score / s_len
        score_vector4[d][s] = score / (d_len*s_len)

108it [04:32,  2.52s/it]


In [51]:
import numpy

project_sdg_df1 = pd.DataFrame(data=score_vector1)
project_sdg_df2 = pd.DataFrame(data=score_vector2)
project_sdg_df3 = pd.DataFrame(data=score_vector3)
project_sdg_df4 = pd.DataFrame(data=score_vector4)

columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']

project_sdg_df1.columns = columns
project_sdg_df2.columns = columns
project_sdg_df3.columns = columns
project_sdg_df4.columns = columns

#threshold1 = numpy.argsort(project_sdg_df1)['sdg17']
#threshold2 = numpy.argsort(project_sdg_df2)['sdg17']
#threshold3 = numpy.argsort(project_sdg_df3)['sdg17']
#threshold4 = numpy.argsort(project_sdg_df4)['sdg17']

In [52]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df1 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df1.T).T)
norm_sdg_df2 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df2.T).T)
norm_sdg_df3 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df3.T).T)
norm_sdg_df4 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df4.T).T)

In [53]:
doc_num = 108
i = 0
match = 0

for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df1.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df2.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df3.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/sdg_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df4.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len*sdg_len): %.2f"% (match/doc_num*100)+"%")

Accuracy Score (sim): 89.81%
Accuracy Score (sim/doc_len): 89.81%
Accuracy Score (sim/sdg_len): 89.81%
Accuracy Score (sim/doc_len*sdg_len): 89.81%


# Similarity (1-gram) Threshold=0.75

In [18]:
documents = data_lemmatized
sdg_l = sdg_list

score_vector1 = np.zeros((len(documents), len(sdg_l)))
score_vector2 = np.zeros((len(documents), len(sdg_l)))
score_vector3 = np.zeros((len(documents), len(sdg_l)))
score_vector4 = np.zeros((len(documents), len(sdg_l)))

for d, doc in tqdm(enumerate(documents)):
    d_len = len(documents[d])
    #print(d)
    for s, sdg in enumerate(sdg_l):
        s_len = len(sdg_l[s])
        score = 0
        for keyword in sdg:
            for word in doc:
                try:
                    sim = model.similarity(word, keyword)
                    if (sim >= 0.75):
                        score += sim 
                except:
                    pass
        score_vector1[d][s] = score
        score_vector2[d][s] = score / d_len
        score_vector3[d][s] = score / s_len
        score_vector4[d][s] = score / (d_len*s_len)

108it [03:36,  2.01s/it]


In [19]:
import numpy

project_sdg_df1 = pd.DataFrame(data=score_vector1)
project_sdg_df2 = pd.DataFrame(data=score_vector2)
project_sdg_df3 = pd.DataFrame(data=score_vector3)
project_sdg_df4 = pd.DataFrame(data=score_vector4)

columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']

project_sdg_df1.columns = columns
project_sdg_df2.columns = columns
project_sdg_df3.columns = columns
project_sdg_df4.columns = columns

#threshold1 = numpy.argsort(project_sdg_df1)['sdg17']
#threshold2 = numpy.argsort(project_sdg_df2)['sdg17']
#threshold3 = numpy.argsort(project_sdg_df3)['sdg17']
#threshold4 = numpy.argsort(project_sdg_df4)['sdg17']

In [20]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df1 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df1.T).T)
norm_sdg_df2 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df2.T).T)
norm_sdg_df3 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df3.T).T)
norm_sdg_df4 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df4.T).T)

In [32]:
doc_num = 108
i = 0
match = 0
cutoff = 0.6

for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df1.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df2.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df3.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
    print (i, np.sum(th1), np.sum(th2))
print("Accuracy Score (sim/sdg_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df4.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len*sdg_len): %.2f"% (match/doc_num*100)+"%")

Accuracy Score (sim): 81.48%
Accuracy Score (sim/doc_len): 81.48%
0 2 1
1 4 4
2 1 3
3 2 4
4 6 3
5 2 1
6 2 2
7 1 3
8 1 4
9 3 1
10 1 4
11 3 3
12 1 2
13 6 3
14 1 1
15 2 3
16 4 3
17 1 1
18 3 1
19 2 6
20 1 4
21 3 2
22 3 2
23 2 1
24 3 2
25 4 1
26 1 2
27 2 6
28 3 2
29 3 2
30 2 4
31 1 2
32 6 5
33 1 2
34 2 1
35 3 2
36 3 3
37 3 3
38 5 1
39 2 2
40 6 2
41 1 2
42 2 2
43 4 3
44 1 2
45 4 1
46 3 2
47 1 2
48 2 1
49 3 3
50 3 2
51 5 2
52 2 3
53 2 1
54 1 1
55 4 5
56 2 3
57 6 2
58 3 2
59 6 1
60 6 1
61 4 1
62 4 2
63 2 1
64 1 1
65 2 1
66 4 3
67 3 4
68 1 3
69 1 2
70 2 4
71 4 1
72 3 4
73 4 2
74 4 2
75 4 2
76 3 2
77 1 3
78 5 4
79 2 1
80 4 5
81 4 1
82 3 3
83 5 2
84 2 3
85 7 3
86 1 3
87 1 3
88 4 2
89 1 3
90 5 3
91 2 2
92 2 6
93 1 1
94 3 2
95 5 3
96 4 2
97 3 1
98 1 5
99 6 2
100 2 2
101 3 3
102 3 6
103 3 2
104 2 2
105 3 1
106 3 3
107 4 1
Accuracy Score (sim/sdg_len): 88.89%
Accuracy Score (sim/doc_len*sdg_len): 88.89%


In [30]:
th1

[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True]

# Similarity (1-gram) Threshold=0.85

In [58]:
documents = data_lemmatized
sdg_l = sdg_list

score_vector1 = np.zeros((len(documents), len(sdg_l)))
score_vector2 = np.zeros((len(documents), len(sdg_l)))
score_vector3 = np.zeros((len(documents), len(sdg_l)))
score_vector4 = np.zeros((len(documents), len(sdg_l)))

for d, doc in tqdm(enumerate(documents)):
    d_len = len(documents[d])
    #print(d)
    for s, sdg in enumerate(sdg_l):
        s_len = len(sdg_l[s])
        score = 0
        for keyword in sdg:
            for word in doc:
                try:
                    sim = model.similarity(word, keyword)
                    if (sim >= 0.85):
                        score += sim 
                except:
                    pass
        score_vector1[d][s] = score
        score_vector2[d][s] = score / d_len
        score_vector3[d][s] = score / s_len
        score_vector4[d][s] = score / (d_len*s_len)

108it [04:28,  2.48s/it]


In [59]:
import numpy

project_sdg_df1 = pd.DataFrame(data=score_vector1)
project_sdg_df2 = pd.DataFrame(data=score_vector2)
project_sdg_df3 = pd.DataFrame(data=score_vector3)
project_sdg_df4 = pd.DataFrame(data=score_vector4)

columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']

project_sdg_df1.columns = columns
project_sdg_df2.columns = columns
project_sdg_df3.columns = columns
project_sdg_df4.columns = columns

#threshold1 = numpy.argsort(project_sdg_df1)['sdg17']
#threshold2 = numpy.argsort(project_sdg_df2)['sdg17']
#threshold3 = numpy.argsort(project_sdg_df3)['sdg17']
#threshold4 = numpy.argsort(project_sdg_df4)['sdg17']

In [60]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df1 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df1.T).T)
norm_sdg_df2 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df2.T).T)
norm_sdg_df3 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df3.T).T)
norm_sdg_df4 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df4.T).T)

In [61]:
doc_num = 108
i = 0
match = 0

for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df1.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df2.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df3.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/sdg_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df4.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len*sdg_len): %.2f"% (match/doc_num*100)+"%")

Accuracy Score (sim): 90.74%
Accuracy Score (sim/doc_len): 90.74%
Accuracy Score (sim/sdg_len): 85.19%
Accuracy Score (sim/doc_len*sdg_len): 85.19%


# Similarity (n-gram) Threshold=0.6

In [62]:
#defining all 17 SDGs
sdg1 = [['africa'],['basic', 'services'],['class'],['developing', 'countries'],['disadvantaged'],['economic', 'resources'],['end', 'poverty'],['environment'],['equality'],['extreme', 'poverty'],['financial', 'inclusion'],['income'],['income', 'equality'],['microfinance'],['non-discrimination'],['poor'],['poor', 'and', 'vulnerable'],['poverty'],['poverty', 'eradication'],['poverty', 'line'],['quality', 'of', 'life'],['resources'],['social', 'protection', 'systems'],['social', 'protection'],['sustainable'],['third', 'world'],['vulnerable'],['wealth', 'distribution']]
sdg2 = [['agricultural', 'orientation', 'index'],['agricultural', 'productivity'],['agriculture'],['consume'],['crop', 'diversity'],['crops'],['doha', 'development', 'round'],['doha', 'round'],['end', 'hunger'],['environment'],['food'],['food', 'gap'],['food', 'production'],['food', 'reserves'],['food', 'security'],['genetic', 'diversity'],['genetic', 'diversity', 'of', 'seeds'],['genetics'],['hunger'],['hungry', 'people'],['improved', 'nutrition'],['innovations', 'and', 'health'],['legumes'],['maize'],['malnourished'],['malnutrition'],['nutrition'],['nutritional', 'needs'],['nutritious'],['poverty'],['produce'],['productivity'],['quality', 'of', 'life'],['resilient', 'agriculture'],['rural', 'infrastructure'],['small', 'scale', 'food', 'producers'],['stunted', 'growth'],['stunting'],['sufficient', 'food'],['sustainable'],['sustainable', 'agriculture'],['sustainable', 'food', 'production'],['trade', 'diversity'],['trade', 'restrictions'],['under', 'nourished'],['undernourished'],['wasting'],['world', 'hungry']]
sdg3 = [['access', 'to', 'clean', 'water', 'and', 'sanitation'],['affordable', 'medicines'],['aids'],['air', 'contamination'],['air', 'pollution'],['alcohol', 'abuse'],['antenatal', 'care'],['antiretroviral'],['antiretroviral', 'therapy'],['biomedical'],['bodily', 'autonomy'],['child', 'deaths'],['contraceptive', 'use'],['death', 'rate'],['dental'],['disability', 'and', 'family', 'support'],['disability', 'and', 'inclusion'],['disability', 'and', 'politics', 'of', 'location'],['diseases'],['family', 'planning'],['health'],['health', 'in', 'resource-constrained', 'settings'],['health', 'worker', 'density'],['healthy'],['healthy', 'lives'],['hepatitis'],['hiv'],['improving', 'mortality'],['increasing', 'life', 'expectancy'],['indigenous'],['infected'],['international', 'health', 'policy'],['international', 'health', 'regulations'],['malaria'],['maternal', 'mortality'],['measles'],['medical'],['mental', 'health'],['mortality'],['mortality', 'rate'],['narcotic', 'drug', 'abuse'],['neonatal', 'mortality'],['polio'],['premature', 'mortality'],['preventable', 'deaths'],['reducing', 'malaria'],['reducing', 'mortality'],['refugees', 'and', 'health', 'services'],['reproductive', 'health'],['road', 'traffic', 'accidents'],['sexual', 'and', 'reproductive', 'health-care'],['sexual', 'health'],['soil', 'contamination'],['soil', 'pollution'],['tobacco', 'control'],['treatment', 'of', 'substance', 'abuse'],['tuberculosis'],['universal', 'health'],['universal', 'health', 'coverage'],['vaccines'],['vaccines', 'in', 'developing', 'countries'],['violence'],['wash'],['water', 'sanitation', 'and', 'hygiene', 'for', 'all'],['water', 'borne', 'disease'],['wellbeing'],['well', 'being'],['well-being'],['world', 'health', 'organisation']]
sdg4 = [['access', 'to', 'education'],['basic', 'education'],['basic', 'literacy'],['basic', 'literacy', 'skills'],['cultural', 'diversity'],['disability'],['disability', 'and', 'education'],['early', 'childhood'],['early', 'childhood', 'development'],['education'],['education', 'for', 'sustainability'],['education', 'in', 'developing'],['enrolment'],['equal', 'access'],['equal', 'education'],['equitable', 'education'],['gender', 'disparities', 'in', 'education'],['gender', 'disparity'],['gender', 'equality'],['gender', 'equity'],['gender', 'sensitive'],['global', 'citizenship'],['global', 'education'],['inclusion', 'and', 'education'],['inclusive'],['innovation'],['international', 'cooperation'],['learning', 'opportunities'],['lifelong', 'learning'],['literacy'],['literacy', 'skills'],['numeracy'],['preprimary', 'education'],['primary', 'education'],['qualified', 'teachers'],['refugees', 'and', 'learning'],['scholarships'],['school'],['school', 'enrolment'],['secondary', 'education'],['teacher', 'training'],['universal', 'education'],['vocational', 'training'],['vulnerable'],['women', 'rights']]
sdg5 = [['basic', 'living', 'standards'],['dignity'],['disadvantaged'],['discrimination'],['employment'],['empower', 'girls'],['empowerment'],['empowerment', 'of', 'women'],['empower', 'women'],['women', 'empowerment'],['equal', 'access'],['equal', 'opportunities'],['equality'],['exploitation'],['female', 'genital', 'mutilation'],['feminism'],['forced', 'marriage'],['gender'],['gender', 'discrimination'],['gender', 'equality'],['gender', 'parity'],['governance', 'and', 'gender'],['human', 'rights'],['human', 'trafficking'],['humanitarian'],['marginalised'],['parity'],['pay'],['reproductive', 'rights'],['sexual', 'and', 'reproductive', 'health'],['sexual', 'exploitation'],['sexual', 'violence'],['social', 'inclusion'],['trafficking'],['universal', 'health', 'coverage'],['violence'],['violence', 'against', 'girls'],['violence', 'against', 'women'],['women'],['women', 'in', 'work'],['women', 'rights'],['workplace', 'equality']]
sdg6 = [['accessible', 'water'],['affordable', 'drinking', 'water'],['aquifer'],['cities'],['clean', 'water'],['contaminated'],['defecation'],['desalination'],['diarrhoeal', 'diseases'],['drought'],['dumping'],['ecosystem', 'protection'],['ecosystem', 'restoration'],['equitable', 'sanitation'],['floods'],['fresh', 'water'],['hydropower'],['hygiene'],['improving', 'water'],['inadequate', 'water'],['inadequate', 'water', 'supply'],['infrastructure'],['irrigation'],['lakes'],['latrines'],['open', 'defecation'],['pollution'],['recycled', 'water'],['reuse'],['river', 'basins'],['rivers'],['safe', 'drinking', 'water'],['sanitation'],['sanitation', 'and', 'hygiene'],['sanitation', 'management'],['sewerage'],['sustainable', 'water', 'management'],['sustainable', 'withdrawals'],['third', 'world'],['toilets'],['untreated', 'wastewater'],['urban'],['waste'],['wastewater'],['wastewater', 'treatment'],['water'],['water', 'access'],['water', 'disasters'],['water', 'ecosystems'],['water', 'efficiency'],['water', 'harvesting'],['water', 'quality'],['water', 'resources', 'management'],['water', 'scarcity'],['water', 'supply'],['water-related', 'ecosystems'],['water-use', 'efficiency']]
sdg7 = [['affordable', 'energy'],['alternative', 'energy'],['animal', 'waste'],['battery'],['carbon'],['charcoal'],['clean', 'energy'],['clean', 'energy', 'technology'],['clean', 'fuel', 'technology'],['clean', 'fuels'],['cleaner', 'fossil', 'fuel', 'technology'],['climate', 'goal'],['coal'],['electricity'],['electricity', 'infrastructure'],['emissions'],['energy'],['energy', 'efficiency'],['energy', 'infrastructure'],['energy', 'research'],['energy', 'technology'],['fossil-fuel'],['green', 'economy'],['greenhouse', 'gas'],['greenhouse', 'gas', 'emissions'],['hydroelectric'],['low', 'carbon'],['modern', 'electricity'],['modern', 'energy'],['reliable', 'energy'],['renewable'],['renewable', 'energy'],['renewable', 'power'],['solar'],['solar', 'energy'],['solar', 'power'],['sustainable', 'energy'],['sustainable', 'energy', 'services'],['sustainable', 'power'],['vehicles'],['wave'],['wind'],['wind', 'power'],['wind', 'turbine'],['wood']]
sdg8 = [['aid', 'for', 'trade'],['banking'],['child', 'labour'],['child', 'soldiers'],['creativity', 'and', 'innovation'],['culture'],['decent', 'work'],['decent', 'work', 'for', 'all'],['development', 'oriented', 'policy'],['economic', 'growth'],['economic', 'productivity'],['economy'],['enterprises'],['entrepreneurship'],['equal', 'pay'],['finance'],['financial', 'services'],['forced', 'labour'],['gdp', 'growth'],['global', 'resource', 'efficiency'],['global', 'trade'],['gross', 'domestic', 'product', 'growth'],['human', 'trafficking'],['inclusive', 'economic', 'growth'],['innovation'],['insurance'],['job', 'creation'],['jobs'],['labour', 'market'],['labour', 'rights'],['micro', 'finance'],['migrant', 'workers'],['modern', 'slavery'],['poverty', 'eradication'],['poverty', 'line'],['productive', 'employment'],['productivity'],['public', 'policy'],['quality', 'jobs'],['quality', 'of', 'life'],['resource', 'efficiency'],['safe', 'work'],['secure', 'work'],['slavery'],['social', 'policies'],['society'],['stable', 'employment'],['stable', 'jobs'],['sustainable', 'consumption'],['sustainable', 'economic', 'growth'],['sustainable', 'production'],['sustainable', 'tourism'],['trade'],['unemployment'],['well-paid', 'jobs'],['women', 'migrants'],['work'],['work', 'opportunities'],['world', 'trade'],['youth', 'employment'],['youth', 'unemployment']]
sdg9 = [['access', 'to', 'the', 'internet'],['affordable', 'access'],['affordable', 'credit'],['clean', 'technologies'],['cooperation'],['data', 'banks'],['economic', 'development'],['electrical', 'power'],['energy'],['enterprises'],['environmentally', 'sound', 'technologies'],['financial', 'services'],['ict', 'infrastructure'],['industrial', 'diversification'],['industrialisation'],['information', 'and', 'communication', 'technology'],['infrastructure'],['innovation'],['internet', 'access'],['irrigation'],['knowledge', 'in', 'education', 'for', 'all'],['mobile', 'networks', 'in', 'developing', 'countries'],['national', 'security'],['network', 'infrastructure'],['phone', 'service'],['public', 'policy'],['quality', 'of', 'life'],['regional', 'infrastructure'],['research'],['resilient', 'infrastructure'],['resource', 'use', 'efficiency'],['roads'],['sanitation'],['scientific', 'research'],['society'],['sustainable', 'industrialisation'],['sustainable', 'infrastructure'],['technological', 'capabilities'],['technology'],['trade'],['transborder', 'infrastructure'],['transport'],['value', 'chains'],['value', 'chains', 'and', 'markets'],['water', 'infrastructure'],['water', 'resources']]
sdg10 = [['affordable', 'housing'],['age'],['ageism'],['business'],['children'],['culture'],['developing', 'countries'],['developing', 'states'],['development', 'assistance'],['disabilities'],['discrimination'],['discriminatory'],['economy'],['education'],['empower'],['equal', 'opportunity'],['equality'],['equity'],['ethnicity'],['financial', 'assistance'],['foreign', 'aid'],['foreign', 'investment'],['gender'],['global', 'financial', 'markets'],['health'],['homelessness'],['homophobia'],['human', 'rights'],['inclusion'],['income', 'growth'],['income', 'inequality'],['indigenous'],['inequalities'],['inequality'],['migrant', 'remittance'],['migration'],['population', 'growth'],['poverty'],['public', 'policy'],['quality', 'of', 'life'],['race'],['racisim'],['reduce', 'inequalities'],['religion'],['rural'],['sex'],['sexism'],['social', 'protection'],['society'],['vulnerable', 'nations'],['world', 'trade']]
sdg11 = [['adaptable'],['adaptation'],['affordable', 'housing'],['air', 'pollution'],['air', 'quality'],['cities'],['climate', 'change'],['community'],['cultural', 'heritage'],['decentralisation'],['development', 'planning'],['disaster', 'management'],['disaster', 'risk', 'reduction'],['disaster', 'strategy'],['disasters'],['fine', 'particulate', 'matter'],['green', 'spaces'],['heritage'],['housing'],['human', 'settlements'],['impact', '', 'of', 'cities'],['inadequate', 'housing'],['informal', 'settlements'],['infrastructure'],['land'],['land', 'consumption'],['local', 'materials'],['mitigation'],['natural', 'disasters'],['natural', 'heritage'],['over', 'crowding'],['pollution'],['population'],['population', 'growth'],['public', 'spaces'],['public', 'transport'],['resilient'],['resilient', 'buildings'],['resource', 'efficiency'],['resource', 'needs'],['risk', 'reduction', 'strategy'],['road', 'safety'],['safe', 'cities'],['shanty'],['slums'],['smart', 'cities'],['solid', 'waste'],['suburban'],['sustainable'],['sustainable', 'building'],['sustainable', 'buildings'],['sustainable', 'cities/city'],['sustainable', 'communities'],['sustainable', 'urbanisation'],['town', 'planning'],['transport'],['transport', 'systems'],['urban'],['urban', 'development'],['urban', 'planning'],['urban', 'sustainability'],['urbanisation'],['waste'],['waste', 'generation'],['waste', 'management'],['water'],['water', 'related', 'disasters']]
sdg12 = [['capitalism'],['cars'],['circular', 'economy'],['commercial', 'enterprises'],['consumer', 'levels'],['consumerism'],['consumption'],['deep', 'decarbonisation'],['ecological'],['efficient', 'use', 'of', 'resources'],['energy'],['energy', 'consumption'],['energy', 'efficiency'],['energy', 'use'],['food'],['food', 'losses'],['food', 'supply'],['food', 'waste'],['fossil', 'fuel', 'subsidies'],['future', 'proof'],['global', 'food', 'waste'],['greenhouse', 'gasses'],['harvest', 'losses'],['life', 'cycle'],['market', 'distortions'],['materialism'],['materials', 'goods'],['monitoring', 'sustainable', 'development'],['natural', 'resources'],['obsolescence'],['overconsumption'],['production'],['recycle'],['recycling'],['reduce', 'waste', 'generation'],['reduction'],['renewable'],['resource', 'efficiency'],['responsible', 'production', 'chains'],['retail'],['retail', 'industry'],['reuse'],['sustainable'],['sustainable', 'consumption'],['sustainable', 'management'],['sustainable', 'practices'],['sustainable', 'production'],['sustainable', 'public', 'procurement'],['sustainable', 'resource', 'use'],['sustainable', 'supply', 'chain'],['sustainable', 'tourism'],['vehicles'],['waste'],['wasteful', 'consumption'],['water'],['water', 'pollution'],['water', 'supply']]
sdg13 = [['average', 'global', 'temperature'],['carbon'],['carbon', 'dioxide'],['changing', 'weather', 'patterns'],['climate'],['climate', 'action'],['climate', 'adaptation'],['climate', 'and', 'gender'],['climate', 'and', 'infectious', 'disease'],['climate', 'and', 'politics'],['climate', 'change'],['climate', 'change', 'management'],['climate', 'change', 'planning'],['climate', 'change', 'policy'],['climate', 'early', 'warning'],['climate', 'hazards'],['climate', 'impact'],['climate', 'mitigation'],['climate', 'refugees'],['climate', 'related', 'hazards'],['climate', 'resilience'],['co2', 'capture'],['co2', 'conversion'],['cop', '21'],['cop', '22'],['ecosystems'],['emissions'],['extreme', 'weather'],['extreme', 'weather', 'events'],['global', 'mean', 'temperature'],['global', 'temperature'],['global', 'warming'],['greenhouse', 'gas'],['greenhouse', 'gas', 'emissions'],['greenhouse', 'gases'],['ice', 'loss'],['low-carbon', 'economy'],['natural', 'disasters'],['natural', 'systems'],['ocean', 'warming'],['paris', 'agreement'],['pollution'],['renewable'],['sea', 'level', 'rise'],['rising', 'sea'],['rising', 'sea', 'level'],['temperature'],['warming']]
sdg14 = [['artisanal', 'fishers'],['biodiversity'],['carbon', 'dioxide'],['coastal', 'biodiversity'],['coastal', 'ecosystems'],['coastal', 'habitats'],['coastal', 'parks'],['coastal', 'resources'],['coastlines'],['conserve'],['conserve', 'oceans'],['coral', 'bleaching'],['coral', 'reef'],['ecosystem', 'management'],['fish', 'species'],['fish', 'stocks'],['fish', 'stocks', 'and', 'fisheries', 'management'],['fisheries'],['fishers'],['fishing'],['fishing', 'practices'],['global', 'warming'],['illegal', 'fishing'],['kelp'],['law', 'of', 'the', 'sea'],['marine'],['marine', 'areas'],['marine', 'biodiversity'],['marine', 'ecosystems'],['marine', 'fisheries'],['marine', 'parks'],['marine', 'pollution'],['marine', 'resources'],['ocean'],['ocean', 'acidification'],['ocean', 'temperature'],['oceanography'],['oceans'],['overfishing'],['productive', 'oceans'],['protected', 'areas'],['sea', 'grasses'],['seas'],['sustainable', 'ecosystems'],['unregulated', 'fishing'],['water', 'resources', 'and', 'policy']]
sdg15 = [['afforestation'],['agriculture'],['animals'],['arable', 'land'],['bees'],['biodiversity'],['biodiversity', 'loss'],['conservation'],['deforestation'],['desertification'],['drought'],['drylands'],['ecosystem'],['ecosystem', 'restoration'],['ecosystems'],['extinct'],['extinct', 'species'],['extinction'],['forest'],['forest', 'management'],['forests'],['genetic', 'resources'],['illegal', 'wildlife', 'products'],['illicit', 'trafficking'],['indigenous'],['indigenous', 'populations'],['invasive', 'alien', 'species'],['land', 'conservation'],['land', 'degradation'],['land', 'loss'],['land', 'use', 'and', 'sustainability'],['manage', 'forests'],['managed', 'forests'],['micro-organisms'],['permaculture'],['plants'],['poaching'],['poverty'],['protected', 'fauna'],['protected', 'flora'],['protected', 'species'],['reforestation'],['soil'],['soil', 'degradation'],['species'],['strategic', 'plan', 'for', 'biodiversity'],['terrestrial', 'ecosystems'],['threatened', 'species'],['tree'],['tree', 'species'],['wetlands']]
sdg16 = [['abuse'],['accountability'],['accountable', 'institutions'],['arbitrary', 'detention'],['arms'],['arms', 'trafficking'],['birth', 'registration'],['bribery'],['combat', 'terrorism'],['conflict', 'resolution'],['conflicts'],['corruption'],['discrimination'],['education'],['enforced', 'disappearance'],['equal', 'access'],['equity'],['exploitation'],['flow', 'of', 'arms'],['freedom'],['geography', 'of', 'poverty'],['governance'],['hate', 'crime'],['human', 'rights'],['human', 'trafficking'],['illegal', 'arms'],['illicit', 'financial', 'flows'],['inclusion'],['inclusive', 'institutions'],['inclusive', 'societies/society'],['institutions'],['internally', 'displaced'],['judiciary'],['justice'],['justice', 'for', 'all'],['legal', 'identity'],['national', 'security'],['non-violence'],['organized', 'crime'],['paris', 'principles'],['peace'],['peaceful', 'societies'],['physical', 'abuse'],['police'],['prevent', 'violence'],['psychological', 'abuse'],['public', 'policy'],['quality', 'of', 'life'],['representative', 'decision-making'],['rule', 'of', 'law'],['security', 'threats'],['sexual', 'abuse'],['sexual', 'violence'],['stolen', 'assets'],['tax', 'evasion'],['theft'],['torture'],['trafficking'],['transparency'],['unsentenced', 'detainees'],['unstable', 'societies'],['victims', 'of', 'violence'],['violence'],['violence', 'against', 'women', 'and', 'children'],['violence', 'rates'],['weapon', 'seizures']]
sdg17 = [['capacity', 'building'],['civil', 'society', 'partnerships'],['communication', 'technologies'],['debt', 'sustainability'],['development', 'assistance'],['disaggregated', 'data'],['doha', 'development', 'agenda'],['entrepreneurship'],['environmentally', 'sound', 'technologies'],['foreign', 'direct', 'investments'],['fostering', 'innovation'],['free', 'trade'],['fundamental', 'principles', 'of', 'official', 'statistics'],['global', 'partnership'],['global', 'partnership', 'for', 'sustainable', 'development'],['global', 'stability'],['international', 'aid'],['international', 'cooperation'],['international', 'population', 'and', 'housing', 'census'],['international', 'support'],['international', 'support', 'for', 'developing', 'countries'],['knowledge', 'sharing'],['multi-stakeholder', 'partnerships'],['poverty', 'eradication'],['public-private', 'partnerships'],['science', 'cooperation', 'agreements'],['technology', 'cooperation', 'agreements'],['technology', 'transfer'],['weighted', 'tariff', 'average'],['women', 'entrepreneurs'],['world', 'trade', 'organization']]
sdg_list = [sdg1, sdg2, sdg3, sdg4, sdg5, sdg6, sdg7, sdg8, sdg9, sdg10, sdg11, sdg12, sdg13, sdg14, sdg15, sdg16, sdg17]

In [63]:
documents = data_lemmatized
sdg_l = sdg_list

score_vector1 = np.zeros((len(documents), len(sdg_l)))
score_vector2 = np.zeros((len(documents), len(sdg_l)))
score_vector3 = np.zeros((len(documents), len(sdg_l)))
score_vector4 = np.zeros((len(documents), len(sdg_l)))

for d, doc in tqdm(enumerate(documents)):
    d_len = len(documents[d])
    #print(d)
    for s, sdg in enumerate(sdg_l):
        s_len = len(sdg_l[s])
        score = 0
        for keyword in sdg:
            for word in doc:
                a = list()
                a.append(word)
                try:
                    sim = model.n_similarity(a, keyword)
                    if (sim >= 0.6):
                        score += sim 
                except:
                    pass
        score_vector1[d][s] = score
        score_vector2[d][s] = score / d_len
        score_vector3[d][s] = score / s_len
        score_vector4[d][s] = score / (d_len*s_len)

108it [10:53,  6.05s/it]


In [64]:
import numpy

project_sdg_df1 = pd.DataFrame(data=score_vector1)
project_sdg_df2 = pd.DataFrame(data=score_vector2)
project_sdg_df3 = pd.DataFrame(data=score_vector3)
project_sdg_df4 = pd.DataFrame(data=score_vector4)

columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']

project_sdg_df1.columns = columns
project_sdg_df2.columns = columns
project_sdg_df3.columns = columns
project_sdg_df4.columns = columns

#threshold1 = numpy.argsort(project_sdg_df1)['sdg17']
#threshold2 = numpy.argsort(project_sdg_df2)['sdg17']
#threshold3 = numpy.argsort(project_sdg_df3)['sdg17']
#threshold4 = numpy.argsort(project_sdg_df4)['sdg17']

In [65]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df1 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df1.T).T)
norm_sdg_df2 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df2.T).T)
norm_sdg_df3 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df3.T).T)
norm_sdg_df4 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df4.T).T)

In [66]:
doc_num = 108
i = 0
match = 0

for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df1.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df2.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df3.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/sdg_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df4.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len*sdg_len): %.2f"% (match/doc_num*100)+"%")

Accuracy Score (sim): 85.19%
Accuracy Score (sim/doc_len): 85.19%
Accuracy Score (sim/sdg_len): 69.44%
Accuracy Score (sim/doc_len*sdg_len): 69.44%


# Similarity (n-gram) Threshold=0.85

In [67]:
documents = data_lemmatized
sdg_l = sdg_list

score_vector1 = np.zeros((len(documents), len(sdg_l)))
score_vector2 = np.zeros((len(documents), len(sdg_l)))
score_vector3 = np.zeros((len(documents), len(sdg_l)))
score_vector4 = np.zeros((len(documents), len(sdg_l)))

for d, doc in tqdm(enumerate(documents)):
    d_len = len(documents[d])
    #print(d)
    for s, sdg in enumerate(sdg_l):
        s_len = len(sdg_l[s])
        score = 0
        for keyword in sdg:
            for word in doc:
                a = list()
                a.append(word)
                try:
                    sim = model.n_similarity(a, keyword)
                    if (sim >= 0.85):
                        score += sim 
                except:
                    pass
        score_vector1[d][s] = score
        score_vector2[d][s] = score / d_len
        score_vector3[d][s] = score / s_len
        score_vector4[d][s] = score / (d_len*s_len)

108it [10:52,  6.04s/it]


In [68]:
import numpy

project_sdg_df1 = pd.DataFrame(data=score_vector1)
project_sdg_df2 = pd.DataFrame(data=score_vector2)
project_sdg_df3 = pd.DataFrame(data=score_vector3)
project_sdg_df4 = pd.DataFrame(data=score_vector4)

columns = ['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5', 'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13', 'sdg14', 'sdg15', 'sdg16', 'sdg17']

project_sdg_df1.columns = columns
project_sdg_df2.columns = columns
project_sdg_df3.columns = columns
project_sdg_df4.columns = columns

#threshold1 = numpy.argsort(project_sdg_df1)['sdg17']
#threshold2 = numpy.argsort(project_sdg_df2)['sdg17']
#threshold3 = numpy.argsort(project_sdg_df3)['sdg17']
#threshold4 = numpy.argsort(project_sdg_df4)['sdg17']

In [69]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
norm_sdg_df1 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df1.T).T)
norm_sdg_df2 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df2.T).T)
norm_sdg_df3 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df3.T).T)
norm_sdg_df4 = pd.DataFrame(min_max_scaler.fit_transform(project_sdg_df4.T).T)

In [70]:
doc_num = 108
i = 0
match = 0

for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df1.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df2.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df3.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/sdg_len): %.2f"% (match/doc_num*100)+"%")

match=0
for i in range(doc_num):
    # use for top-3
    #th1 = list(project_sdg_df.iloc[i] >= project_sdg_df.iloc[i][columns[threshold[i]]])
    
    #use for score > 0.7
    th1 = list(norm_sdg_df4.iloc[i] >= cutoff)
    th2 = list(df2.iloc[i])
    found = np.dot(th1, th2)
    if found > 0:
        match += 1
print("Accuracy Score (sim/doc_len*sdg_len): %.2f"% (match/doc_num*100)+"%")

Accuracy Score (sim): 82.41%
Accuracy Score (sim/doc_len): 82.41%
Accuracy Score (sim/sdg_len): 76.85%
Accuracy Score (sim/doc_len*sdg_len): 76.85%
